## Chroma

In [1]:
# !pip install -qU chromadb

Chroma는 다양한 모드로 실행될 수 있습니다. 아래에서 각각의 예시를 확인할 수 있습니다.

- in-memory - Python 스크립트 또는 Jupyter 노트북에서 실행
- in-memory with persistance - 스크립트 또는 노트북에서 실행하고 디스크에 저장/로드
- in a docker container - 로컬 머신이나 클라우드에서 서버로 실행

다른 데이터베이스와 마찬가지로 다음과 같은 작업을 수행할 수 있습니다:

- .add
- .get
- .update
- .upsert
- .delete
- .peek
- .query: 유사도 검색을 실행합니다

### 기본 사용법

- TextLoader를 사용하여 문서를 로드합니다.
- CharacterTextSplitter를 사용하여 로드된 문서를 청크 단위로 분할합니다.
- SentenceTransformerEmbeddings를 사용하여 오픈 소스 임베딩 함수를 생성합니다. 모델로는 "all-MiniLM-L6-v2"를 사용합니다.
- Chroma 벡터 스토어에 분할된 문서와 임베딩 함수를 로드합니다.
- 질의(query) 를 사용하여 Chroma 벡터 스토어에서 유사도 검색을 수행합니다.
- 검색 결과 중 가장 유사한 문서의 내용을 출력합니다.

In [2]:
# import
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# 문서를 로드하고 청크로 분할합니다.
loader = TextLoader("./data/appendix-keywords.txt")
documents = loader.load()

# 문서를 청크로 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 오픈 소스 임베딩 함수를 생성합니다.
stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Chroma에 로드합니다.
db = Chroma.from_documents(docs, stf_embeddings)

# 질의합니다.
query = "What is Word2Vec?"
docs = db.similarity_search(query)

# 결과를 출력합니다.
print(docs[0].page_content)

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


### 로컬 디스크 공간에 저장

- Chroma는 데이터를 자동으로 디스크에 저장하기 위해 최선을 다하지만, 여러 개의 인메모리 클라이언트가 서로의 작업을 중단시킬 수 있습니다. 따라서, 한 번에 경로당 하나의 클라이언트만 실행하는 것이 좋습니다.
- Chroma.from_documents() 메서드를 사용하여 docs와 embedding_function으로부터 Chroma 데이터베이스를 생성하고, persist_directory를 지정하여 디스크에 저장합니다.
- db2.similarity_search() 메서드를 사용하여 query와 유사한 문서를 검색합니다.
- Chroma 클래스의 생성자를 사용하여 persist_directory에 저장된 Chroma 데이터베이스를 로드하고, embedding_function을 지정합니다.
- db3.similarity_search() 메서드를 사용하여 query와 유사한 문서를 검색합니다.
- 검색된 문서 중 첫 번째 문서의 page_content를 출력합니다.

In [3]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
db2 = Chroma.from_documents(docs, stf_embeddings, persist_directory=DB_PATH)

# 질의합니다.
query = "What is Word2Vec?"
docs = db2.similarity_search(query)

# 결과를 출력합니다.
docs[0]

Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)')

In [4]:
# 디스크에서 문서를 로드합니다.
db3 = Chroma(persist_directory="./chroma_db",
             embedding_function=stf_embeddings)

# 질의합니다.
query = "What is Word2Vec?"
docs = db3.similarity_search(query)
print(docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


### Langchain API - Chroma Client 활용

- chromadb 모듈을 사용하여 PersistentClient를 생성합니다.
- PersistentClient를 사용하여 "chroma_collection"이라는 이름의 컬렉션을 가져오거나 생성합니다.
- collection.add() 메서드를 사용하여 컬렉션에 문서를 추가합니다. 문서의 ID와 내용을 각각 "1", "2", "3"과 "a", "b", "c"로 지정합니다.
- Chroma 클래스를 사용하여 langchain_chroma 객체를 생성합니다. 이 객체는 PersistentClient, 컬렉션 이름, 임베딩 함수를 매개변수로 받습니다.
- langchain_chroma._collection.count() 메서드를 사용하여 컬렉션의 문서 수를 출력합니다.

In [ ]:
import chromadb

# ChromaDB의 PersistentClient를 생성합니다.
persistent_client = chromadb.PersistentClient()
# "my_chroma_collection"이라는 이름의 컬렉션을 가져오거나 생성합니다.
collection = persistent_client.get_or_create_collection("chroma_collection")
# 컬렉션에 ID와 문서를 추가합니다.
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

# Chroma 객체를 생성합니다.
langchain_chroma = Chroma(
    # PersistentClient를 전달합니다.
    client=persistent_client,
    # 사용할 컬렉션의 이름을 지정합니다.
    collection_name="chroma_collection",
    # 임베딩 함수를 전달합니다.
    embedding_function=stf_embeddings,
)

# 컬렉션의 항목 수를 출력합니다.
print(
    "현재 저장된 Collection 의 개수는 ",
    langchain_chroma._collection.count(),
    " 개 입니다.",
)

C:\Users\anHye\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:24<00:00, 3.33MiB/s]


현재 저장된 Collection 의 개수는  3  개 입니다.


### 업데이트 & 삭제

- 문서 개수에 해당하는 간단한 ID 리스트를 생성합니다.
- Chroma.from_documents() 메서드를 사용하여 문서, 임베딩 함수, ID를 전달하고 Chroma 데이터베이스를 생성합니다.
- similarity_search() 메서드를 사용하여 쿼리와 유사한 문서를 검색하고, 첫 번째 문서의 메타데이터를 출력합니다.
- 첫 번째 문서의 메타데이터를 업데이트하고, update_document() 메서드를 사용하여 데이터베이스에 반영합니다.
- 업데이트된 문서의 정보를 출력합니다.
- 마지막 문서를 삭제하기 전후의 문서 개수를 출력하여 삭제 작업을 확인합니다.

In [6]:
# 간단한 ID 생성
ids = [str(i) for i in range(1, len(docs) + 1)]

# 데이터 추가
example_db = Chroma.from_documents(docs, stf_embeddings, ids=ids)
docs = example_db.similarity_search(query)
print(docs[0].metadata)

# 문서의 메타데이터 업데이트
docs[0].metadata = {
    "source": "./images/appendix-keywords.txt",
    "new_value": "테스트용으로 업데이트할 내용입니다.",
}

# DB 에 업데이트
example_db.update_document(ids[0], docs[0])
print(example_db._collection.get(ids=[ids[0]]))

# 문서 개수 출력
print("count before", example_db._collection.count())
# 마지막 문서 삭제
example_db._collection.delete(ids=[ids[-1]])
# 삭제 후 문서 개수 출력
print("count after", example_db._collection.count())

{'source': './data/appendix-keywords.txt'}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'new_value': '테스트용으로 업데이트할 내용입니다.', 'source': './images/appendix-keywords.txt'}], 'documents': ['정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)'], 'uris': None, 'data': None, 'included': ['metadatas', 'documents']}
count before 34
count after 33


### OpenAI Embeddings 사용하기

In [7]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

- chromadb.EphemeralClient를 사용하여 새로운 Chroma 클라이언트를 생성합니다.
- Chroma.from_documents 메서드를 사용하여 문서(docs)와 임베딩(embedings)을 기반으로 Chroma 컬렉션을 생성합니다.
    - 새로 생성된 클라이언트(new_client)를 사용합니다.
    - 컬렉션 이름은 "openai_collection"으로 지정합니다.
- 질의(query)를 정의합니다: "Word2Vec 에 대해서 설명해 주세요."
- openai_lc_client.similarity_search 메서드를 사용하여 질의와 유사한 문서를 검색합니다.

In [8]:
from langchain_openai import OpenAIEmbeddings


# 문서를 로드하고 청크로 분할합니다.
loader = TextLoader("./data/appendix-keywords.txt")
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# OpenAIEmbeddings 를 사용
openai_embeddings = OpenAIEmbeddings()
# client 생성
new_client = chromadb.EphemeralClient()


# OpenAI 임베딩과 Chroma 클라이언트를 사용하여 문서에서 Chroma 검색 클라이언트를 생성합니다.
openai_lc_client = Chroma.from_documents(
    docs, openai_embeddings, client=new_client, collection_name="openai_collection"
)

query = "Word2Vec 에 대해서 설명해 주세요."
# 질의를 사용하여 유사도 검색을 수행합니다.
docs = openai_lc_client.similarity_search(query)
# 검색 결과에서 첫 번째 문서의 내용을 출력합니다.
print(docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


#### 유사도 검색과 유사도 점수 계산

코사인 유사도를 산정합니다. score가 높을 수록 유사도가 높다고 판단할 수 있습니다.

- db 객체의 similarity_search_with_score 메서드를 사용하여 query와 유사한 문서를 검색하고, 그 결과를 docs 변수에 저장합니다.

In [9]:
query = "Word2Vec 에 대해서 설명해 주세요."
# 유사도 검색을 수행하고 점수와 함께 결과를 반환합니다.
docs = db.similarity_search_with_score(query)

In [10]:
# 문서 리스트의 첫 번째 문서를 가져옵니다.
document, score = docs[0]
print(document, end="\n\n")
print(f"유사도 점수: {score}")

page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)' metadata={'source': './data/appendix-keywords.txt'}

유사도 점수: 0.8870157599449158


#### 검색기(Retriever)

#### MMR

retriever 객체에서 유사도 검색을 사용하는 것 외에도 mmr을 사용할 수 있습니다.

- db 객체를 사용하여 retriever를 생성합니다.
- search_type 매개변수를 "mmr"로 설정하여 MMR(Maximal Marginal Relevance) 검색 알고리즘을 사용합니다.

In [11]:
# 검색 유형을 "mmr"로 설정하여 데이터베이스를 검색기로 사용
retriever = db.as_retriever(search_type="mmr")

- retriever 객체의 get_relevant_documents() 메서드를 사용하여 주어진 query와 관련된 문서를 검색합니다.
- get_relevant_documents() 메서드는 관련 문서의 리스트를 반환하며, 이 중 첫 번째 문서([0])를 선택합니다.

In [12]:
query = "Word2Vec 에 대해서 설명해 주세요."
# 질의와 관련된 문서 중 가장 관련성이 높은 문서를 가져옵니다.
retriever.get_relevant_documents(query)[0]

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)')

### 메타데이터를 기반으로 필터링

In [13]:
# metadatas 중 chunk_id 기준으로 필터링
example_db.get(where={"chunk_id": 0})

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [14]:
# page_number 기준으로 필터링
example_db.get(where={"page_number": 0})

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [15]:
# source 기준으로 필터링
example_db.get(where={"source": "./data/appendix-keywords.txt"})

{'ids': ['00655209-0633-4a12-b5e9-ab8d0e25c6c7',
  '01caed30-8204-456f-8e96-f9c29f038264',
  '060852aa-5f4d-4672-9f84-96630775694c',
  '0b565d6e-9ee6-483a-a17d-691bb80a47c6',
  '12449504-3a05-40a3-a829-bafb24a66de1',
  '2',
  '24c81640-14f5-4615-8076-3640da730d53',
  '3',
  '4520b053-091d-4195-a386-2bb8cf6d33ff',
  '51baf9e9-a74e-42ed-9cb1-608d14288f55',
  '5e24ec16-552c-4dc6-a1ce-234c3cb5e00b',
  '6b968e4f-8c75-4855-9466-f210a2649864',
  '71cb3aa3-499b-413d-8106-2107c91cdb4d',
  '7d0c4faa-7041-403e-8c10-eec3f510176e',
  '83c7e8b9-bf6c-41a8-aa72-1a0891dd8979',
  '89f4e4a6-e24d-40cc-930c-bdbdd5c5cb8f',
  '8bd8e7bc-a0c3-4caf-bf75-80295b74f459',
  '9879f459-d168-4ff8-9feb-4d638b4f832b',
  'a3b147f7-bd28-4fe7-9e85-c8f82c86cc3a',
  'a722b26b-4a7d-49d3-9043-b53570e2a0ca',
  'aa0a9a67-b86e-4ba4-92aa-799f87f1faf9',
  'bd7368be-5371-4fd3-939a-6011fb9e5888',
  'c1d675fc-850e-490d-8c70-317a0b76fd1d',
  'd47466cb-e647-4bf6-a978-3deabf6c2e1e',
  'db0c403e-2ddf-40cb-a1d2-4da3ab36cd6e',
  'e2777cc0-b